In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
# importing Natural Language Toolkit 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
pd.options.mode.chained_assignment = None

In [5]:
data = pd.read_csv("../../../Data/consumer-reviews-of-amazon-products/Complete.csv",low_memory=False)

In [6]:
data =  data[['brand','categories','manufacturer','reviews.username','reviews.title','reviews.text','reviews.numHelpful',
                   'reviews.rating']]

In [7]:
data.head()

,brand,categories,manufacturer,reviews.username,reviews.title,reviews.text,reviews.numHelpful,reviews.rating
0,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,Adapter,Kindle,This product so far has not disappointed. My c...,0.0,5.0
1,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,truman,very fast,great for beginner or experienced person. Boug...,0.0,5.0
2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,DaveZ,Beginner tablet for our 9 year old son.,Inexpensive tablet for him to use and learn on...,0.0,5.0
3,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,Shacks,Good!!!,I've had my Fire HD 8 two weeks now and I love...,0.0,4.0
4,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",Amazon,explore42,Fantastic Tablet for kids,I bought this for my grand daughter when she c...,0.0,5.0


In [8]:
data['reviews.clean'] = data['reviews.text'].str.replace('\d+', '')
data = data.dropna()

In [9]:
from nltk.corpus import stopwords
stopset = set(stopwords.words("english"))
vectorizer = TfidfVectorizer(stop_words=stopset,binary=True)

In [10]:
X = vectorizer.fit_transform(data['reviews.clean'])
y = data['reviews.rating'].map({1.0:0,2.0:0,3.0:0,4.0:1,5.0:1})

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, train_size=0.80, random_state=42)

In [12]:
# Import the models from sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# Initialize the three models
model = MultinomialNB(alpha=1.0,fit_prior=True)

model.fit(X_train,y_train)
print(f1_score(y_test,model.predict(X_test)))

0.9657471961200363


In [13]:
import pickle
from sklearn.externals import joblib

# save vertorizer information
with open('TfidfVectorizerModel.pkl','wb') as TfidfVectorizerModel:
    pickle.dump(vectorizer.vocabulary_, TfidfVectorizerModel)

# save trained model
with open('MultinomialNBModel.pkl','wb') as MultinomialNBModel:
    joblib.dump(model, MultinomialNBModel)

In [28]:
%%writefile score.py
import json, os, pickle
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

vocab =  pickle.load(open('TfidfVectorizerModel.pkl', 'rb'))
model = joblib.load(open('MultinomialNBModel.pkl','rb'))

transformer = TfidfTransformer()
trainedVectorizer = CountVectorizer(decode_error='replace',vocabulary=vocab)

def run(raw_data):
    vectorString  = trainedVectorizer.fit_transform([str(data)])
    transformedString = transformer.fit_transform(vectorString)
    y_hat = model.predict(transformedString)
    return(str(y_hat))

Overwriting score.py
